In [ ]:

# Install required libraries
!pip install openai
!apt-get install ffmpeg -y
!pip install -U openai-whisper
!pip install openai chromadb sentence-transformers


import warnings
warnings.simplefilter("ignore", category=FutureWarning)


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


In [ ]:
import whisper
import openai
import subprocess
import os
from datetime import datetime
import textwrap
import chromadb
from sentence_transformers import SentenceTransformer

# Set OpenAI API key directly
openai.api_key = "..."
# Initialize OpenAI client
client = openai.OpenAI(api_key=openai.api_key)

In [ ]:
try:
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": "who is the prime minister of spain"}],
        max_tokens=5
    )
    print(response.choices[0].message.content)
except Exception as e:
    print("API Key is invalid or there's an issue:", e)

As of my last knowledge


In [ ]:
# Specify video path and parameters
video_file = "/content/sap-class.mp4"
model_size = "small"  # Options: "tiny", "small", "medium", "large"
save_transcript = True

print(f"Using video file: {video_file}")
print(f"Whisper model size: {model_size}")
print(f"Save transcript: {save_transcript}")

# Verify file existence
if os.path.exists(video_file):
    print("File verified.")
else:
    print("Warning: File not found. Check path or re-upload.")

Using video file: /content/sap-class.mp4
Whisper model size: small
Save transcript: True
File verified.


In [ ]:
# Global variables to store results
transcript = None

try:
    # Load Whisper model
    print(f"Loading Whisper '{model_size}' model...")
    model = whisper.load_model(model_size)

    # Transcribe video with timing
    print(f"Transcribing video {video_file}...")
    start_time = datetime.now()
    result = model.transcribe(video_file, verbose=False)
    transcript = result["text"]
    end_time = datetime.now()
    print(f"Transcription completed in {(end_time - start_time).total_seconds():.2f} seconds.")

    # Print and optionally save transcript
    print("\nTranscription:\n", transcript)
    if save_transcript:
        transcript_file = f"/content/transcript_{os.path.basename(video_file)}.txt"
        with open(transcript_file, "w") as f:
            f.write(transcript)
        print(f"Transcript saved to: {transcript_file}")
except FileNotFoundError as e:
    print(f"Error: {e}")
except Exception as e:
    print(f"Unexpected Error: {e}")

Loading Whisper 'small' model...


100%|███████████████████████████████████████| 461M/461M [00:06<00:00, 69.1MiB/s]


Transcribing video /content/sap-class.mp4...


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Detected language: Telugu


  3%|▎         | 11854/439049 [10:28<6:17:24, 18.86frames/s]


KeyboardInterrupt: 

In [ ]:
# Global variable to store summary
summary = None

try:
    if transcript is None:
        raise ValueError("No transcript available. Run the transcription cell first.")

    # Generate summary with language detection and translation
    print("\nGenerating summary and checking for other languages...")
    summary_prompt = (
        "You are an expert summarizer and translator."
        "Create a detailed, concise summary with most important details mentioned in the class of the following transcript."
        "Detect if any languages like Telugu, Hindi, or others are used alongside English. "
        "If detected, translate those portions into English and include them in the summary. "
        f"{transcript}\n\nSummary:"
    )
    summary_response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are an expert summarizer and translator specializing in multilingual text."},
            {"role": "user", "content": summary_prompt}
        ],
        max_tokens=300,
        temperature=0.5
    )
    summary = summary_response.choices[0].message.content.strip()
    print("\nSummary:\n", summary)
except Exception as e:
    print(f"Unexpected Error: {e}")

In [ ]:
# Cell 2: Initialize SentenceTransformer and ChromaDB client
model = SentenceTransformer("all-MiniLM-L6-v2")
chroma_client = chromadb.PersistentClient(path="chroma_db")

In [ ]:
# Cell 3: Check and delete collection if it exists
if any(c == "ml_mcqs" for c in chroma_client.list_collections()):
    chroma_client.delete_collection("ml_mcqs")

existing_collections = chroma_client.list_collections()

if "ml_mcqs" in existing_collections:
    chroma_client.get_collection("ml_mcqs").delete()

In [ ]:
# Cell 4: Create a new collection
collection = chroma_client.create_collection("ml_mcqs")

In [ ]:
# Cell 5: Extract text from the summary
# Assuming summary variable is defined elsewhere with your ML content
# summary = "Your machine learning text content here..."
full_text = summary  # Use your defined summary variable

In [ ]:
# Cell 6: Split text into chunks
text_chunks = textwrap.wrap(full_text, 500)

In [ ]:
for i, chunk in enumerate(text_chunks):
    embedding = model.encode(chunk)
    collection.add(
        ids=[f"chunk_{i}"],
        embeddings=[embedding.tolist()],
        metadatas=[{"text": chunk}]
    )


In [ ]:
# Cell 8: Generate MCQs using OpenAI
all_mcqs = []
num_questions = 5

for chunk in text_chunks:
    prompt = f"""
    Based on the following text, generate up to {num_questions} unique multiple-choice questions.
    Each question should have 4 answer choices, the correct answer (a/b/c/d), and a brief explanation.

    Text:
    {chunk}

    Format:
    1. [Generated Question]
       a) Option 1
       b) Option 2
       c) Option 3
       d) Option 4
       Correct Answer: [a/b/c/d]
       Explanation: [Brief explanation]
    """

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}]
    )

    if response.choices:
        mcqs_text = response.choices[0].message.content.strip()
    else:
        mcqs_text = ""

    mcqs_list = mcqs_text.split("\n\n")
    all_mcqs.extend(mcq for mcq in mcqs_list if mcq not in all_mcqs)  # Ensure uniqueness


In [ ]:
# Cell 9: Store generated MCQs in ChromaDB
unique_mcqs = all_mcqs[:num_questions]
for i, mcq in enumerate(unique_mcqs):
    embedding = model.encode(mcq)
    collection.add(
        ids=[f"mcq_{i}"],
        embeddings=[embedding.tolist()],
        metadatas=[{"question": mcq}]
    )

print(f"Generated {len(unique_mcqs)} MCQs and stored in ChromaDB!")


In [ ]:
user_answers = {}
print("\n=== Begin Test (20 Questions) ===")
for i, mcq in enumerate(unique_mcqs, 1):
    # Display only the question and options, hide answer and explanation
    lines = mcq.split("\n")
    question_part = "\n".join(line for line in lines if not line.strip().startswith("Correct Answer") and not line.strip().startswith("Explanation"))
    print(f"\nQ{i}: {question_part}")
    answer = input("Your Answer (a/b/c/d): ").strip().lower()
    while answer not in ['a', 'b', 'c', 'd']:
        answer = input("Invalid input! Please enter a/b/c/d: ").strip().lower()
    user_answers[i-1] = answer

score = 0
print("\n=== Test Results ===")

for i, (mcq, user_ans) in enumerate(zip(unique_mcqs, user_answers.values())):
    lines = mcq.split("\n")
    # Extract question part (excluding Correct Answer and Explanation)
    question_part = "\n".join(line for line in lines if not line.strip().startswith("Correct Answer") and not line.strip().startswith("Explanation"))

    # Extract correct answer safely
    correct_answer_line = next((line for line in lines if line.strip().startswith("Correct Answer")), None)
    correct_answer = correct_answer_line.split(": ")[1].strip().lower() if correct_answer_line else "Unknown"

    # Extract explanation safely
    explanation_line = next((line for line in lines if line.strip().startswith("Explanation")), None)
    explanation = explanation_line.split(": ")[1].strip() if explanation_line else "No explanation provided"

    # Display results
    print(f"\nQ{i+1}: {question_part}")
    print(f"Your Answer: {user_ans}")
    print(f"Correct Answer: {correct_answer}")
    print(f"Explanation: {explanation}")

    # Compare answers and update score
    if user_ans == correct_answer:
        print("✅ Correct!")
        score += 1
    else:
        print("❌ Incorrect!")

print(f"\nFinal Score: {score}/20 ({score * 5}%)")

In [ ]:
# prompt: i want to take the above test of the user on some platform and send him the mail to take the exam and in return after evaluation send him and the mentor his score back

import smtplib
from email.mime.text import MIMEText
import random
import string

def send_test_mail(sender_email, sender_password, receiver_email, test_link):
    msg = MIMEText(f"Please click on the following link to take your test:\n {test_link}")
    msg["Subject"] = "Test Invitation"
    msg["From"] = sender_email
    msg["To"] = receiver_email

    with smtplib.SMTP_SSL("smtp.gmail.com", 465) as smtp:
      smtp.login(sender_email, sender_password)
      smtp.send_message(msg)


def send_results_mail(sender_email, sender_password, receiver_email, mentor_email, score):
    msg_user = MIMEText(f"Your test score is: {score}")
    msg_user["Subject"] = "Test Results"
    msg_user["From"] = sender_email
    msg_user["To"] = receiver_email

    msg_mentor = MIMEText(f"The test score for {receiver_email} is: {score}")
    msg_mentor["Subject"] = "Student Test Results"
    msg_mentor["From"] = sender_email
    msg_mentor["To"] = mentor_email

    with smtplib.SMTP_SSL("smtp.gmail.com", 465) as smtp:
      smtp.login(sender_email, sender_password)
      smtp.send_message(msg_user)
      smtp.send_message(msg_mentor)


# Example usage
sender_email = "huzaif027@gmail.com"
sender_password = "xfet hunl pxnk myzz"
receiver_email = "huzaif0207@gmail.com"
mentor_email = "mihawk6927@gmail.com"

test_link = "https://www.lancieretechnologies.com"

# Send the test invitation email
send_test_mail(sender_email, sender_password, receiver_email, test_link)

score = score

# Send the results emails after evaluation
send_results_mail(sender_email, sender_password, receiver_email, mentor_email, score)

print("Emails sent successfully!")


In [ ]:
import smtplib
from email.mime.text import MIMEText

def send_test_mail(sender_email, sender_password, receiver_email, test_link):
    msg = MIMEText(
        f"🚨 Attention! 🚨\n\nYou've been selected for the ultimate test! 🧠🔥\nClick here to prove your intelligence (or expose your lack of it 😆):\n\n{test_link}\n\nNo pressure… but i am watching. 👀"
    )
    msg["Subject"] = "📝 Your *Very Serious* Test Invitation"
    msg["From"] = sender_email
    msg["To"] = receiver_email

    with smtplib.SMTP_SSL("smtp.gmail.com", 465) as smtp:
        smtp.login(sender_email, sender_password)
        smtp.send_message(msg)


def send_results_mail(sender_email, sender_password, receiver_email, mentor_email, score):
    msg_user = MIMEText(
        f"📢 BREAKING NEWS! 📢\n\nYour test results are in… and the results are *shocking*! 🫣\n\nScore: {score}\n\nDon't worry, we still like you (kinda). 😆"
    )
    msg_user["Subject"] = "🎉 Your Glorious Test Results"
    msg_user["From"] = sender_email
    msg_user["To"] = receiver_email

    msg_mentor = MIMEText(
        f"🚨 Urgent Update 🚨\n\nWe have received the latest test results for {receiver_email}.\n\nScore: {score}\n\nWe suggest offering them *a motivational speech* or perhaps *a tissue*. 😅"
    )
    msg_mentor["Subject"] = "📊 Student Test Results - Somewhat Concerning"
    msg_mentor["From"] = sender_email
    msg_mentor["To"] = mentor_email

    with smtplib.SMTP_SSL("smtp.gmail.com", 465) as smtp:
        smtp.login(sender_email, sender_password)
        smtp.send_message(msg_user)
        smtp.send_message(msg_mentor)


# Example usage
sender_email = "huzaif027@gmail.com"
sender_password = "xfet hunl pxnk myzz"  # Make sure to use environment variables for real use
receiver_email = "huzaif0207@gmail.com"
mentor_email = "mihawk6927@gmail.com"

test_link = "https://www.lancieretechnologies.com"

# Send the test invitation email
send_test_mail(sender_email, sender_password, receiver_email, test_link)

score = "0 🤡 (but hey, at least you tried!)"

# Send the results emails after evaluation
send_results_mail(sender_email, sender_password, receiver_email, mentor_email, score)

